# BigQuery のテーブルに自然言語で問い合わせる

## はじめに
Google が提供する生成 AI の Gemini を利用し、BIgQueryに自然言語で問い合わせる方法を理解します。

ここでは、以下の2通りの方法を確認します。

1.   BigQuery のデータを Pandas としてロードし、生成 AI に Dataframe を扱うコードを生成、実行させる
2.   生成 AI に BigQuery を検索する SQL を生成、実行させる

フレームワークとしては、LangChain を利用しています。

この処理を、Google Cloud の Vertex AI を利用して確認します。

## 環境セットアップ

前提パッケージを導入します。

In [ ]:
# Install Vertex AI LLM SDK
! pip install langchain langchain-experimental langchain-google-vertexai sqlalchemy-bigquery --upgrade --user

**※ 注意: ここでカーネルを再起動します。**

* Colab の場合、上記のログに"RESTART RUNTIME"ボタンをが表示された場合、ボタンを押してカーネルをリスタートできます。
* Vertex AI Workbench の場合、メニューよりカーネルのリスタートを実行できます。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

続いて、Google Cloud でプロジェクトを作成し Vertex AI API を有効化します。

また、このコードを実行するユーザーに`Vertex AI ユーザー`、`BigQuery ジョブユーザー`、`BigQuery データ閲覧者`、`BigQuery 読み取りセッション ユーザー`のロールを付与します。

Colab の場合、以下を実行し Vertex AI API のユーザー権限をもつアカウントでログインします。 Vertex AI Workbench の場合はスキップされます。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

環境変数などを定義します。 Google Cloud のプロジェクト ID、BigQuery の テーブル情報などを指定してください。

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type:"string"}
BQ_LOCATION = "asia-northeast1"  # @param {type:"string"}
BQ_DATASET = "<your_dataset>"  # @param {type:"string"}
BQ_TABLE = "<your_table>"  # @param {type:"string"}
REGION = "us-central1"

Vertex AI と LangChain のライブラリーの導入を確認します。 LangChain を利用して Gemini モデルを取得します。

In [ ]:
import langchain
from google.cloud import aiplatform

print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

from langchain_google_vertexai import VertexAI
# Text model instance integrated with LangChain
llm = VertexAI(
    model_name="gemini-pro",
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

## Pandas (BigQuery DataFrames) を利用した検索

BigQuery DataFrames は Pandas と同じ API を提供するライブラリです。ここでは、Pandas  を利用してテーブルの照会を確認します。

In [ ]:
import bigframes.pandas as bpd


# Set BigQuery DataFrames options
bpd.close_session()
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = BQ_LOCATION

# Create a DataFrame from a BigQuery table
table = PROJECT_ID+"."+BQ_DATASET+"."+BQ_TABLE
bdf = bpd.read_gbq(table)
bdf.head(5)

Pandas を処理する LangChain Agent を呼び出します。bigframes.dataframe.DataFrame は、通常の pd.DataFrame のインスタンスではないため LangChain のコードをカスタマイズします。また、プロンプトの安定性を向上させるためデフォルトのプロンプトを微修正しています。

In [ ]:
from typing import Any, Union
from langchain.agents import create_react_agent
from langchain.agents.agent import AgentExecutor, BaseSingleActionAgent, BaseMultiActionAgent, RunnableAgent
from langchain.prompts import PromptTemplate
from langchain.schema.language_model import BaseLanguageModel
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

AGENT_TEMPLATE = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always check the Obervation and think about what to do next
Action: the tool you should use is always "{tool_names}"
Action Input: the pandas dataframe command
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`, you should always use these keys for query. :
{df_head}
Do not use df.query() method.

Begin!
Question: {input}
{agent_scratchpad}

"""

# hack Langchain as bigframes.dataframe.DataFrame is not instance of pd.DataFrame
def hack_create_pandas_dataframe_agent(
    llm: BaseLanguageModel,
    df: Any,
    verbose: bool = False,
) -> AgentExecutor:

    tools = [PythonAstREPLTool(locals={"df": df})]
    prompt = PromptTemplate(template=AGENT_TEMPLATE, input_variables=["input", "agent_scratchpad", "df_head"])
    partial_prompt = prompt.partial()
    partial_prompt = partial_prompt.partial(df_head=str(df.head(5).to_markdown()))
    agent: Union[BaseSingleActionAgent, BaseMultiActionAgent] = RunnableAgent(
        runnable=create_react_agent(llm, tools, partial_prompt),  # type: ignore
        input_keys_arg=["input"],
        return_keys_arg=["output"],
    )
    new_agent = AgentExecutor(
            agent=agent,
            tools=tools,
            verbose=verbose,
            return_intermediate_steps=verbose,
            max_iterations=10,
            early_stopping_method="force",
    )
    return new_agent

#agent = create_pandas_dataframe_agent(llm, bdf, verbose=True)
agent = hack_create_pandas_dataframe_agent(llm, bdf, verbose=True)

対象テーブルの件数を確認する質問を投げてみます。

In [ ]:
agent.invoke("レコードは何件ありますか")


対象テーブルに関する質問を投げてみます。

In [ ]:
response = agent.invoke("""
東京都杉並区の中で地価を高い順に5個教えてください。
以下のフォーマットで表形式で回答してください。
[最寄り駅] [住所] [地価]
""")

response の形式を確認します。

In [ ]:
response

pandas のコードを取り出してみます。

In [ ]:
#最後に成功した場合のコマンド
command = response["intermediate_steps"][len(response["intermediate_steps"])-1][0].tool_input
command

pandas のコードが正しく実行できるか確認します。

In [ ]:
tool = PythonAstREPLTool(locals={"df": bdf})
tool.invoke(command)

別の質問を投げてみます。

In [ ]:
response = agent.invoke("""
"1住居"の土地の中から、地価の値上がり上位3件を教えてください。
以下のフォーマットで表形式で回答してください。
[最寄り駅] [住所] [地価]
""")

もう１つ別の質問を試してみます。

In [ ]:
agent.invoke("""
二子玉川 駅の近くで、区分"1住居"の地価の平均は？
""")

## SQL を生成させる検索

今度は、SQL を生成させるパターンを試します。

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit


db = SQLDatabase.from_uri("bigquery://"+PROJECT_ID+"/"+BQ_DATASET)
sqlagent = create_sql_agent(llm=llm, toolkit=SQLDatabaseToolkit(db=db, llm=llm), verbose=True, max_iterations=10)
#sqlagent = SQLDatabaseChain.from_llm(llm, db, verbose=True)

対象テーブルの件数を確認する質問を投げてみます。

In [ ]:
sqlagent.invoke("地価の情報は何件ありますか？")

対象テーブルに関する質問を投げてみます。

In [ ]:
response = sqlagent.invoke("""
東京都杉並区の中で地価を高い順に5つ教えてください。
以下のフォーマットで表形式で回答してください。
[最寄り駅] [住所] [地価]
""")
response

以上、ありがとうございました。

## 参考情報
- [LangChain](https://python.langchain.com/docs/get_started/introduction.html)
- [Overview of Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)